### HAM 10000

* [Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T)

* [Original Paper](https://www.nature.com/articles/sdata2018161)

In [1]:
%cd ..

/home/datascience/Data Fusion


### Setup Environment:

In [2]:
import os
import pandas as pd

from src.classifiers import process_labels, split_data
from src.classifiers_base import preprocess_df

from transformers import BertTokenizer

from src.multimodal_data_loader import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_base_cpu_metrics import calculate_memory

In [3]:
PATH = 'datasets/ham10000/'

In [4]:
text_path = os.path.join(PATH, 'labels.csv')
images_path = os.path.join(PATH, 'images')

## Get data

In [5]:
df = pd.read_csv(text_path)
df

,image_id,dx,text,split
0,ISIC_0033319,nv,Patient diagnosed via histo. Age: 35 years. Se...,train
1,ISIC_0030823,nv,Patient diagnosed via follow_up. Age: 40 years...,train
2,ISIC_0028730,akiec,Patient diagnosed via histo. Age: 65 years. Se...,train
3,ISIC_0027299,nv,Patient diagnosed via follow_up. Age: 40 years...,train
4,ISIC_0032444,nv,Patient diagnosed via histo. Age: 65 years. Se...,train
...,...,...,...,...
10010,ISIC_0034116,nv,Patient diagnosed via histo. Age: 35 years. Se...,test
10011,ISIC_0026453,bcc,Patient diagnosed via histo. Age: 55 years. Se...,test
10012,ISIC_0029885,mel,Patient diagnosed via histo. Age: 35 years. Se...,test
10013,ISIC_0033226,mel,Patient diagnosed via histo. Age: 65 years. Se...,test


## Data Perparation

In [6]:
# Select features and labels vectors
text_columns = 'text'
image_columns = 'image_id'
label_columns = 'dx'

df = preprocess_df(df, image_columns, images_path)

# Split the data
train_df, test_df = split_data(df)

# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)

100%|██████████| 10015/10015 [00:04<00:00, 2238.79it/s]


Train Shape: (8012, 4)
Test Shape: (2003, 4)


In [7]:
train_df

,image_id,dx,text,split
0,datasets/ham10000/images/ISIC_0033319.jpg,nv,Patient diagnosed via histo. Age: 35 years. Se...,train
1,datasets/ham10000/images/ISIC_0030823.jpg,nv,Patient diagnosed via follow_up. Age: 40 years...,train
2,datasets/ham10000/images/ISIC_0028730.jpg,akiec,Patient diagnosed via histo. Age: 65 years. Se...,train
3,datasets/ham10000/images/ISIC_0027299.jpg,nv,Patient diagnosed via follow_up. Age: 40 years...,train
4,datasets/ham10000/images/ISIC_0032444.jpg,nv,Patient diagnosed via histo. Age: 65 years. Se...,train
...,...,...,...,...
8007,datasets/ham10000/images/ISIC_0032692.jpg,vasc,Patient diagnosed via histo. Age: 35 years. Se...,train
8008,datasets/ham10000/images/ISIC_0029317.jpg,nv,Patient diagnosed via follow_up. Age: 45 years...,train
8009,datasets/ham10000/images/ISIC_0025983.jpg,nv,Patient diagnosed via histo. Age: 20 years. Se...,train
8010,datasets/ham10000/images/ISIC_0027256.jpg,vasc,Patient diagnosed via consensus. Age: 0 years....,train


In [8]:
# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Models

In [10]:
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [11]:
calculate_memory(train_loader, test_loader, output_size)

Early fusion:
Average Memory per Batch in Train: 36.51 MB
Total Memory Usage per Epoch Train: 4600.85 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 9.13 MB
Total Memory Usage per Epoch Test: 292.12 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.95 MB

Late fusion:
Average Memory per Batch in Train: 36.51 MB
Total Memory Usage per Epoch Train: 4600.85 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 9.13 MB
Total Memory Usage per Epoch Test: 292.12 MB (excluding model parameters)
Model: 
Model Memory Usage: 747.57 MB
